In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import warnings
warnings.filterwarnings("ignore")
import TinySleepNet
import TinySleepNetCopy
import wandb
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import DatasetsPaco

In [2]:
# wandb.init()

In [3]:
binary_labels = True
channels = ['F4']
batch_size = 32

In [4]:
train_dataset = DatasetsPaco.EDFData_GEN_TF("/home/jorgevi/SSD_IA3/Data/PSG1.edf", channels=channels)

In [5]:
def our_generator():
    for x, y in train_dataset:
        yield x, y

train_dataset_generator = tf.data.Dataset.from_generator(our_generator, output_types=(tf.float64, tf.int32), output_shapes=(tf.TensorShape([15360,1]),tf.TensorShape([]))).batch(32).shuffle(32)

next(iter(train_dataset_generator))

(<tf.Tensor: shape=(32, 15360, 1), dtype=float64, numpy=
 array([[[ 1.36846788],
         [ 0.89248439],
         [ 0.24341599],
         ...,
         [ 1.84445138],
         [ 0.71939949],
         [-0.03784698]],
 
        [[-1.03308519],
         [-1.94178095],
         [-2.35285761],
         ...,
         [ 1.64973086],
         [ 2.40697732],
         [ 1.54155279]],
 
        [[ 0.67612826],
         [-0.34074557],
         [-0.96817835],
         ...,
         [ 0.76267071],
         [ 1.43337472],
         [ 2.60169784]],
 
        ...,
 
        [[-2.80720549],
         [-2.37449322],
         [-1.59561114],
         ...,
         [-0.27583873],
         [-1.09799203],
         [-2.20140831]],
 
        [[-2.13650147],
         [-1.29271255],
         [-0.29747434],
         ...,
         [ 0.32995845],
         [ 1.04393369],
         [ 2.40697732]],
 
        [[ 1.3901035 ],
         [ 0.32995845],
         [-0.86000029],
         ...,
         [ 1.75790892],
         [ 0.

In [6]:
test_dataset = DatasetsPaco.EDFData_GEN_TF("/home/jorgevi/SSD_IA3/Data/PSG2.edf", channels=channels)

In [7]:
def our_test_generator():
    for x, y in test_dataset:
        yield x, y

test_dataset_generator = tf.data.Dataset.from_generator(our_test_generator, output_types=(tf.float64, tf.int32), output_shapes=(tf.TensorShape([15360,1]),tf.TensorShape([]))).batch(32).shuffle(32)

In [8]:
val_dataset = DatasetsPaco.EDFData_GEN_TF("/home/jorgevi/SSD_IA3/Data/PSG3.edf", channels=channels)

In [9]:
def our_val_generator():
    for x, y in val_dataset:
        yield x, y

val_dataset_generator = tf.data.Dataset.from_generator(our_val_generator, output_types=(tf.float64, tf.int32), output_shapes=(tf.TensorShape([15360,1]),tf.TensorShape([]))).batch(32).shuffle(32)

In [10]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(128, 512//2, strides=512//4, input_shape=(15360, 1), activation="relu"),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1, activation="sigmoid")
        # tf.keras.layers.Dense(2, activation="softmax")
    ])
# <=
# model = TinySleepNetCopy.TinySleepNet(512, len(channels), 2)

In [11]:
# gradientape model

train_loss_results = []
train_accuracy_results = []
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.BinaryCrossentropy() # <= from logits = True, if SparseCategoricalCrossentropy()

In [12]:
# for epoch in range(10):

#     epoch_loss_avg = tf.keras.metrics.Mean()
#     epoch_accuracy = tf.keras.metrics.BinaryCrossentropy()

#     for x, y in train_dataset_generator:
#         # print(x.shape, y)
#         with tf.GradientTape() as tape:
#             y_ = model(x)
#             y__ = tf.squeeze(y_, axis = 1) # (64,1) => (64,)
#             # print('y {}, y__{}'.format(y.shape, y__.shape))
#             loss = loss_fn(y, y__)
#         grads = tape.gradient(loss, model.trainable_variables)
#         # wandb.log({'gradients': grads})
#         optimizer.apply_gradients(zip(grads, model.trainable_variables))
#         # Track progress
#         epoch_loss_avg.update_state(loss)  # loss = loss.numpy()
#         epoch_accuracy.update_state(y, y_)
#         # break

#     # End epoch
#     train_loss_results.append(epoch_loss_avg.result())
#     train_accuracy_results.append(epoch_accuracy.result())
#     # wandb.log({'epoch_loss': epoch_loss_avg.result().numpy(), 'epoch_accuracy': epoch_accuracy.result().numpy()})
#     print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
#                                                                 epoch_loss_avg.result(),
#                                                                 epoch_accuracy.result()))

In [13]:
# test_accuracy = tf.keras.metrics.Accuracy()
# for x, y in tftestdatasetmemo:
#     logits = model(x)
#     predictions = tf.argmax(logits, axis=1, output_type=tf.int32)
#     test_accuracy(predictions, y)
# print("Test set accuracy: {:.3%}".format(test_accuracy.result()))

In [14]:
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
# model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
history = model.fit(train_dataset_generator, epochs=50, validation_data=val_dataset_generator)
model.summary()

plt.plot(history.history['accuracy'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['val_loss'])
plt.xlabel('epochs')
plt.legend(['accuracy', 'loss', 'val_accuracy', 'val_loss'], loc='upper left')
plt.savefig('train')

Epoch 1/50
28/28 [==============================] - 7s 116ms/step - loss: 1.0246 - accuracy: 0.5283 - val_loss: 0.7709 - val_accuracy: 0.9555
Epoch 2/50
28/28 [==============================] - 7s 127ms/step - loss: 0.9895 - accuracy: 0.6429 - val_loss: 5.9623 - val_accuracy: 0.0445
Epoch 3/50
28/28 [==============================] - 5s 102ms/step - loss: 1.0102 - accuracy: 0.6939 - val_loss: 0.3463 - val_accuracy: 0.9531
Epoch 4/50
28/28 [==============================] - 6s 111ms/step - loss: 0.9543 - accuracy: 0.5986 - val_loss: 1.0635 - val_accuracy: 0.6272
Epoch 5/50
28/28 [==============================] - 6s 106ms/step - loss: 0.5850 - accuracy: 0.7426 - val_loss: 0.3219 - val_accuracy: 0.8628
Epoch 6/50
28/28 [==============================] - 6s 107ms/step - loss: 0.5754 - accuracy: 0.7166 - val_loss: 0.2211 - val_accuracy: 0.9555
Epoch 7/50
28/28 [==============================] - 6s 110ms/step - loss: 0.5069 - accuracy: 0.7971 - val_loss: 0.2039 - val_accuracy: 0.9496
Epoch 

In [25]:
# test <=
y_ = model.predict(test_dataset_generator)

In [26]:
y_pred = np.array(y_.shape[0]*[1])
y_pred[np.where(y_<0.5)[0]] = 0

y_real = []
for x, y in test_dataset:
    y_real.append(y)
y_real = np.array(y_real) 
# <= mas eficientemente, sin cargarlo en memo?

In [27]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
cm = confusion_matrix(y_real, y_pred)
cm

array([[420, 402],
       [ 12,   6]])

In [28]:
m = tf.keras.metrics.binary_accuracy(y_real, y_pred, threshold=0.5)
print("Which predictions match with binary labels:", m.numpy())
# m = tf.keras.metrics.BinaryAccuracy()
# m.update_state(y_real, y_pred)
# print("Binary Accuracy: ", m.result().numpy())

Which predictions match with binary labels: 0.50714284
